In [28]:
import re

import numpy as np
import pandas as pd
import pymysql
from sshtunnel import SSHTunnelForwarder

In [32]:
SSH_PKEY_PATH = "~/.ssh/vook-rails-ssh-key-dev.pem"
# 対応表を読み出し
ng_ok_table = pd.read_csv("../data/input/query_ng_ok.csv")

In [35]:
def get_ec2_config():
    return {
        "host_name": "3.114.154.242",
        "ec2_port": 22,
        "ssh_username": "ec2-user",
        "ssh_pkey": "~/.ssh/vook-rails-ssh-key.pem",
        "rds_end_point": "vook-rails-db.ctutkiavfpne.ap-northeast-1.rds.amazonaws.com",
        "rds_port": 3306,
    }


def read_sql_file(file_path):
    """
    指定されたファイルパスからSQLファイルを読み込み、その内容を文字列として返す。

    :param file_path: 読み込む.sqlファイルのパス
    :return: ファイルの内容を含む文字列
    """
    try:
        with open(file_path, "r") as file:
            return file.read()
    except IOError as e:
        # ファイルが開けない、見つからない、などのエラー処理
        return f"Error reading file: {e}"


def get_knowledges():
    config_ec2 = get_ec2_config()
    query = read_sql_file("../vook_db_v7/sql/knowledges.sql")
    df_from_db = pd.DataFrame()
    with SSHTunnelForwarder(
        (config_ec2["host_name"], config_ec2["ec2_port"]),
        ssh_username=config_ec2["ssh_username"],
        ssh_pkey=config_ec2["ssh_pkey"],
        remote_bind_address=(
            config_ec2["rds_end_point"],
            config_ec2["rds_port"],
        ),
    ) as server:
        print(f"Local bind port: {server.local_bind_port}")
        conn = None
        try:
            conn = pymysql.connect(
                **get_rds_config(server.local_bind_port), connect_timeout=10
            )
            cursor = conn.cursor()
            cursor.execute(query)
            for (
                row
            ) in (
                cursor
            ):  # column1, column2, ...は取得したいカラム名に合わせて変更してください
                df_from_db = pd.concat(
                    [df_from_db, pd.DataFrame([row])], ignore_index=True
                )
            return df_from_db
        except pymysql.MySQLError as e:
            print(f"Error connecting to MySQL: {e}")
        finally:
            if conn is not None:
                conn.close()


def get_rds_config(port):
    return {
        "user": "root",
        "password": "rds-vook",
        "port": port,
        "host": "localhost",
        "database": "vook_web_v3_production",
        "charset": "utf8mb4",
        "cursorclass": pymysql.cursors.DictCursor,
    }


def create_wort_list(df_from_db: pd.DataFrame, unit: str) -> list:
    """brand,line,knowledgeの連続2文字以上ワードかどうかを判定、修正する"""
    words = df_from_db[
        f"{unit}_name"
    ].values.copy()  # NOTE:copyしないと関数内部で_nameカラムが更新される。
    for row in np.arange(len(words)):
        word = words[row]
        words[row] = validate_input(word)
    return list(words)


def convertor(input_string, ng_ok_table):
    # 特定のワードが DataFrame に含まれているかどうかを確認し、行番号を表示
    row_indices = ng_ok_table.index[
        ng_ok_table.apply(lambda row: input_string in row.values, axis=1)
    ].tolist()
    if row_indices:
        output = ng_ok_table["corrected"][row_indices[0]]
        print(f"{input_string}を{output}に変換します")
        return output
    else:
        print(f"{input_string}は対応表に存在しません。")
        return input_string


def validate_input(input_string):
    """
    連続する2文字以上で構成されたワードのみをOKとし、単体1文字またはスペースの前後に単体1文字が含まれるワードをNGとするバリデータ関数
    """
    # 正規表現パターン: 単体1文字またはスペースの前後に単体1文字が含まれるワードを検出
    pattern_ng = re.compile(r"^[!-~]$|\s[!-~]$|^[!-~]\s")
    # 入力文字列がNGパターンに一致するか確認
    if not pattern_ng.search(input_string):
        return input_string
    else:
        # エラーワードがあればメッセージを吐き、convertor関数によって対応する
        print(f"エラーワード　{input_string}が存在しました:")
        return convertor(input_string, ng_ok_table)


def create_df_no_ng_keyword(
    df_from_db, words_knowledge_name, words_brand_name, words_line_name
):
    df_no_ng_keyword = pd.DataFrame(columns=df_from_db.columns)
    df_no_ng_keyword["knowledge_id"] = df_from_db["knowledge_id"].values
    df_no_ng_keyword["knowledge_name"] = words_knowledge_name
    df_no_ng_keyword["brand_name"] = words_brand_name
    df_no_ng_keyword["line_name"] = words_line_name
    return df_no_ng_keyword


def create_api_input() -> pd.DataFrame:
    # 知識情報の取得
    df_from_db = get_knowledges()
    # 対象のワードリスト作成
    words_brand_name = create_wort_list(df_from_db, "brand")
    words_line_name = create_wort_list(df_from_db, "line")
    words_knowledge_name = create_wort_list(df_from_db, "knowledge")
    # 修正版のテーブルを作成
    df_api_input = create_df_no_ng_keyword(
        df_from_db, words_knowledge_name, words_brand_name, words_line_name
    )
    return df_api_input

In [36]:
# APIのインプットデータ作成
df_api_input = create_api_input()

Local bind port: 64530
エラーワード　BIG Eが存在しました:
BIG EをBIGEに変換します
エラーワード　BIG Eが存在しました:
BIG EをBIGEに変換します
エラーワード　BIG Eが存在しました:
BIG EをBIGEに変換します
エラーワード　BIG Eが存在しました:
BIG EをBIGEに変換します


In [37]:
df_api_input

,knowledge_id,knowledge_name,brand_name,line_name
0,5,66前期,Levi's,501
1,8,1st,U.S.ARMY,M-65 FIELD JACKET
2,10,BIGE,Levi's,501
3,11,片面タブ,Levi's,501XX
4,12,両面タブ,Levi's,501XX
...,...,...,...,...
111,145,71205,Levi's,71205
112,147,ガスプロテクティブコート,U.S.ARMY,コート U.S.ARMY
113,148,517,Levi's,スタプレ
114,149,44-J,Lee,44-J
